In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# %pip install sdv
# %pip install -U matplotlib

# Import modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from sdv.tabular import CTGAN
from sdv.tabular import TVAE
from sdv import Metadata
from sdmetrics.reports import utils
from sdmetrics.reports.single_table import QualityReport

# Data Preprocessing

In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/SI670 Final_Proj/transactions.csv")

In [ ]:
df.head()

,OBJECTID,AGENCY,TRANSACTION_DATE,TRANSACTION_AMOUNT,VENDOR_NAME,VENDOR_STATE_PROVINCE,MCC_DESCRIPTION
0,481,Department of Youth Rehabilitation Services,2009-01-06T00:00:00.000Z,977.00,MY SAFE FOOD HANDLER,MD,Business Services Not Elsewhere Classified
1,482,Department of Youth Rehabilitation Services,2009-01-06T00:00:00.000Z,-25.61,AMES SAFETY ENVELOPE C,MA,Miscellaneous Publishing And Printing
2,483,DC Public Library,2009-01-07T00:00:00.000Z,1035.29,WWW.AIA.ORG,DC,"Membership Organizations, Not Elsewhere Classi..."
3,484,Office of the State Superintendent of Education,2009-01-06T00:00:00.000Z,222.40,DELTA 00675274330522,TN,Delta
4,485,Office of the State Superintendent of Education,2009-01-06T00:00:00.000Z,222.40,DELTA 00675274330500,TN,Delta


In [ ]:
df['Date'] = df['TRANSACTION_DATE'].apply(lambda x: x.split('T')[0])

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df_2009 = df[df['Date'].dt.year==2009].drop(['OBJECTID',
                                             'VENDOR_STATE_PROVINCE',
                                             'TRANSACTION_DATE',
                                             'VENDOR_NAME',
                                             'MCC_DESCRIPTION'], axis=1)
df_2009.head()

,AGENCY,TRANSACTION_AMOUNT,Date
0,Department of Youth Rehabilitation Services,977.00,2009-01-06
1,Department of Youth Rehabilitation Services,-25.61,2009-01-06
2,DC Public Library,1035.29,2009-01-07
3,Office of the State Superintendent of Education,222.40,2009-01-06
4,Office of the State Superintendent of Education,222.40,2009-01-06


In [ ]:
df_2009.isnull().sum()

AGENCY                0
TRANSACTION_AMOUNT    0
Date                  0
dtype: int64

In [ ]:
df_2009.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28235 entries, 0 to 42999
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   AGENCY              28235 non-null  object        
 1   TRANSACTION_AMOUNT  28235 non-null  float64       
 2   Date                28235 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 882.3+ KB


In [ ]:
len(df_2009)

28235

In [ ]:
df_2009.to_csv('real_data.csv', index=False)

# CTGAN

In [ ]:
import time
start_time = time.time()

ctgan = CTGAN()
ctgan.fit(df_2009)

end_time = time.time()
print(f"total run time = {(end_time - start_time)/60} minutes")

In [ ]:
ctgan.save('ctgan_model.pkl')

In [ ]:
ctgan_loaded = CTGAN.load('ctgan_model.pkl')

In [ ]:
ctgan_syn_data = ctgan_loaded.sample(num_rows=28235)

In [ ]:
ctgan_syn_data.head()

In [ ]:
ctgan_syn_data.to_csv('ctgan_syn_data.csv', index=False)

In [ ]:
ctgan_syn_data = pd.read_csv('/content/drive/Shareddrives/SI670 Final_Proj/synthetic_samples/ctgan_syn_data.csv')
ctgan_syn_data

,AGENCY,TRANSACTION_AMOUNT,Date
0,Office of the Chief Financial Officer,2585.43,2009-09-21
1,Department of Health Care Finance,12.86,2009-08-11
2,Department of Health,1156.49,2009-11-29
3,Office of the Attorney General,2066.25,2009-08-16
4,District Department of Transportation,153.16,2009-08-21
...,...,...,...
28230,Department of Housing & Community Development,2002.56,2009-01-21
28231,Department on Disability Services,2358.65,2009-04-19
28232,Department of Public Works,2422.49,2009-04-19
28233,Office of the Secretary,22.41,2009-04-07


In [ ]:
ctgan_syn_data['Date'] = pd.to_datetime(ctgan_syn_data['Date'])

In [ ]:
metadata = {
    "fields": {
        "AGENCY": {
            "type": "categorical"
            },
        "TRANSACTION_AMOUNT": {
            "type": "numerical",
            "subtype": "float"
        },
        "Date": {
            "type": "datetime",
            "format": "%Y-%m-%d"
        }
    }
}

In [ ]:
fig1 = utils.get_column_plot(
    real_data=df_2009,
    synthetic_data=ctgan_syn_data,
    column_name='AGENCY',
    metadata=metadata
)

fig1.show()

In [ ]:
fig2 = utils.get_column_plot(
    real_data=df_2009,
    synthetic_data=ctgan_syn_data,
    column_name='TRANSACTION_AMOUNT',
    metadata=metadata
)

fig2.show()

In [ ]:
fig3 = utils.get_column_plot(
    real_data=df_2009,
    synthetic_data=ctgan_syn_data,
    column_name='Date',
    metadata=metadata
)

fig3.show()

/usr/local/lib/python3.8/dist-packages/sdmetrics/reports/utils.py:117: FutureWarning:

casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.

/usr/local/lib/python3.8/dist-packages/sdmetrics/reports/utils.py:118: FutureWarning:

casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.



In [ ]:
report = QualityReport()
report.generate(df_2009, ctgan_syn_data, metadata)
report.get_details(property_name='Column Shapes')

Creating report: 100%|██████████| 4/4 [00:00<00:00, 18.78it/s]



Overall Quality Score: 91.92%

Properties:
Column Shapes: 91.65%
Column Pair Trends: 92.18%


,Column,Metric,Quality Score
0,TRANSACTION_AMOUNT,KSComplement,0.809598
1,Date,KSComplement,0.892793
2,AGENCY,TVComplement,0.981796


# TVAE

In [ ]:
start_time = time.time()
tvae_model = TVAE()
tvae_model.fit(df_2009)
end_time = time.time()
print(f"total run time = {(end_time - start_time)/60} minutes")

In [ ]:
tvae_model.save('tvae_model.pkl')

In [ ]:
tvae_loaded = TVAE.load('tvae_model.pkl')

In [ ]:
tvae_syn_data = tvae_loaded.sample(num_rows=28235)

In [ ]:
tvae_syn_data.head()

In [ ]:
tvae_syn_data.to_csv('tvae_syn_data.csv', index=False)

In [ ]:
tvae_syn_data = pd.read_csv("/content/drive/Shareddrives/SI670 Final_Proj/synthetic_samples/tvae_syn_data.csv")

In [ ]:
tvae_syn_data['Date'] = pd.to_datetime(tvae_syn_data['Date'])

In [ ]:
fig4 = utils.get_column_plot(
    real_data=df_2009,
    synthetic_data=tvae_syn_data,
    column_name='AGENCY',
    metadata=metadata
)

fig4.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig5 = utils.get_column_plot(
    real_data=df_2009,
    synthetic_data=tvae_syn_data,
    column_name='TRANSACTION_AMOUNT',
    metadata=metadata
)

fig5.show()

In [ ]:
fig6 = utils.get_column_plot(
    real_data=df_2009,
    synthetic_data=tvae_syn_data,
    column_name='Date',
    metadata=metadata
)

fig6.show()

/usr/local/lib/python3.8/dist-packages/sdmetrics/reports/utils.py:117: FutureWarning:

casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.

/usr/local/lib/python3.8/dist-packages/sdmetrics/reports/utils.py:118: FutureWarning:

casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.



In [ ]:
report = QualityReport()
report.generate(df_2009, tvae_syn_data, metadata)
report.get_details(property_name='Column Shapes')

Creating report: 100%|██████████| 4/4 [00:00<00:00, 65.40it/s]



Overall Quality Score: 70.3%

Properties:
Column Shapes: 66.0%
Column Pair Trends: 74.59%


,Column,Metric,Quality Score
0,TRANSACTION_AMOUNT,KSComplement,0.726155
1,Date,KSComplement,0.796317
2,AGENCY,TVComplement,0.558845
